In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

/Users/sejal/anaconda3/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:14: UserWarning: A NumPy version >=1.25.2 and <2.6.0 is required for this version of SciPy (detected version 1.24.3)
  from scipy.sparse import csr_matrix, issparse


In [2]:
# Load Sample Dataset
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
x, y = data.data, data.target

In [3]:
df = pd.DataFrame(data.data, columns=data.feature_names)

In [4]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [5]:
# Split Data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
# Feature Scaling (for SVR)
scaler_x = StandardScaler()
scaler_y = StandardScaler()
x_train_scaled = scaler_x.fit_transform(x_train)
x_test_scaled = scaler_x.transform(x_test)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()

In [7]:
# Initialize Models
models = {
    "Linear Regression": LinearRegression(),
    "Polynomial Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "SVR": SVR(kernel='rbf')
}

In [8]:
# Polynomial Feature Transformation
poly = PolynomialFeatures(degree=2)
x_poly_train = poly.fit_transform(x_train)
x_poly_test = poly.transform(x_test)

In [9]:
# Train and Evaluate
results = {}
for name, model in models.items():
    if name == "Polynomial Regression":
        model.fit(x_poly_train, y_train)
        y_pred = model.predict(x_poly_test)
    elif name == "SVR":
        model.fit(x_train_scaled, y_train_scaled)
        y_pred = scaler_y.inverse_transform(
            model.predict(x_test_scaled).reshape(-1, 1)
        ).ravel()
    else:
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)

    results[name] = {
        "MAE": mean_absolute_error(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "R2_Score": r2_score(y_test, y_pred)
    }

In [10]:
# Display Results
results_df = pd.DataFrame(results).T
print(results_df)

                            MAE       MSE  R2_Score
Linear Regression      0.533200  0.555892  0.575788
Polynomial Regression  0.467001  0.464302  0.645682
Decision Tree          0.455762  0.499707  0.618663
Random Forest          0.327599  0.255726  0.804850
SVR                    0.397283  0.354151  0.729740
